<a href="https://colab.research.google.com/github/pradeepti239/DB_withNeon/blob/main/c226f25_DBwithNEONandColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install SQLAlchemy psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.3 MB/s eta 0:00:00


In [2]:
# Simplified query cell (uses session-level search_path)
# - Connect with the STUDENT URL
# - Set search_path so we can omit schema prefixes
# - Run example queries and show results

import os, getpass, pandas as pd
from sqlalchemy import create_engine, text

STUDENT_DB_URL = os.environ.get("NEON_STUDENT_DB_URL") or getpass.getpass("Paste STUDENT DB URL (hidden): ")
engine = create_engine(STUDENT_DB_URL, pool_pre_ping=True, future=True)

# Example 1: quick row counts
with engine.connect() as conn:
    counts_sql = """
    SELECT 'product' AS table, COUNT(*) AS rows FROM product
    UNION ALL SELECT 'pc', COUNT(*) FROM pc
    UNION ALL SELECT 'laptop', COUNT(*) FROM laptop
    UNION ALL SELECT 'printer', COUNT(*) FROM printer;
    """
    counts_df = pd.read_sql_query(text(counts_sql), conn)

# Example 2: laptops with screens > 15" (maker, model, screen, price)
with engine.connect() as conn:
    laptops_sql = """
    SELECT p.maker, l.model, l.screen, l.price
    FROM product p
    JOIN laptop l USING (model)
    WHERE l.screen > 15
    ORDER BY l.screen DESC, l.price DESC;
    """
    laptops_df = pd.read_sql_query(text(laptops_sql), conn)

# Display results
print("Row counts:")
display(counts_df)

print("\nLaptops (screen > 15\"):")
display(laptops_df)


Paste STUDENT DB URL (hidden): ··········
Row counts:


,table,rows
0,product,30
1,pc,13
2,laptop,10
3,printer,7



Laptops (screen > 15"):


,maker,model,screen,price
0,E,2001,20.1,3673
1,A,2005,17.0,2500
2,E,2002,17.0,949
3,E,2003,16.4,549
4,G,2010,15.4,2300
5,A,2006,15.4,1700
6,F,2008,15.4,900


In [3]:
# Example 3: model where speed is greater than 3
with engine.connect() as conn:
    counts_sql = """
      select model from pc where speed >= 3.00;
    """
    result = pd.read_sql_query(text(counts_sql), conn)

# Display results
print("Results:")
display(result)



Results:


,model
0,1005
1,1006
2,1013


In [11]:
# Example 4: Display maker and price of PC having ram size greater than or equal to 1024
with engine.connect() as conn:
    counts_sql = """
      SELECT p.maker, p.model, pc.ram, pc.price FROM
      product p JOIN pc ON p.model = pc.model
      WHERE pc.ram >=1024;
    """
    result = pd.read_sql_query(text(counts_sql), conn)

# Display results
print("Results:")
display(result)

Results:


,maker,model,ram,price
0,A,1001,1024,2114
1,B,1004,1024,649
2,B,1006,1024,1049
3,C,1007,1024,510
4,D,1008,2048,770
5,D,1009,1024,650
6,D,1010,2048,770
7,E,1011,2048,959
8,E,1012,1024,649


In [13]:
# Example 5: Display the maker of Laptops with atleat 100gb.
with engine.connect() as conn:
    counts_sql = """
      SELECT p.maker, p.model, l.speed, l.ram, l.hd, l.screen, l.price FROM
      product p JOIN laptop l on p.model = l.model
      WHERE l.hd>=100;
    """
    result = pd.read_sql_query(text(counts_sql), conn)

# Display results
print("Results:")
display(result)

Results:


,maker,model,speed,ram,hd,screen,price
0,A,2005,2.16,1024,120,17.0,2500
1,B,2007,1.83,1024,120,13.3,1429
2,E,2001,2.00,2048,240,20.1,3673
3,F,2008,1.60,1024,100,15.4,900
4,G,2010,2.00,2048,160,15.4,2300


In [18]:
# Example 6: Display the model number and price of all products made by maker B.
with engine.connect() as conn:
    counts_sql = """
    with product_laptop as(
    SELECT p.maker, p.model, l.price
    FROM product p JOIN laptop l on p.model = l.model
    WHERE p.maker = 'B'
    ),
    product_pc as(
    SELECT p.maker, p.model, pc.price
    FROM product p JOIN pc on p.model = pc.model
    WHERE p.maker = 'B'
    ),
    product_printer as(
    SELECT p.maker, p.model, pr.price
    FROM product p JOIN printer pr on p.model = pr.model
    WHERE p.maker = 'B'
    )
    SELECT model, price FROM product_laptop
    UNION ALL
    SELECT model, price FROM product_pc
    UNION ALL
    SELECT model, price FROM product_printer
    ORDER BY model;

    """
    result = pd.read_sql_query(text(counts_sql), conn)

# Display results
print("Results:")
display(result)

Results:


,model,price
0,1004,649
1,1005,630
2,1006,1049
3,2007,1429
